
# SOCIODEMOGRAPHIC

## sociodemographic.csv
|Variable|Explicación|
|---|---|
|*age*| Edad del cliente
|*country_id*| País de residencia del cliente
|*deceased*| Fallecido. N/S
|*gender*| Sexo del cliente
|*region_code*| Provincia de residencia del cliente (para ES)
|*salary*| Ingresos brutos de la unidad familiar

In [1]:
import numpy as np 
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
pd.set_option('display.max_columns', 100)
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette("Spectral")
import plotly.express as px
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

/Users/claudiacastro/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Cargar el archivo parquet
df_full = pd.read_parquet("C:/Users/Bobby/Desktop/Master Data Analysis/TFM/easyMoney_fintech/data/df_completo.parquet")


# eliminamos columnas que no aportan información
df_full = df_full.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0'], axis=1)

# Ordenar df_full por pk_cid y pk_partition
df_full = df_full.sort_values(by=['pk_cid', 'pk_partition'])

# Modificamos el tipo de dato de las columnas, SIMPLEMENTE CAMBIA EL TIPO DE DATO
df_full["pk_partition"] = pd.to_datetime(df_full["pk_partition"])
df_full["entry_date"] = pd.to_datetime(df_full["entry_date"], errors='coerce')
df_full["entry_channel"] = df_full["entry_channel"].astype('category')
df_full['active_customer'] = df_full['active_customer'].astype('int32')
df_full['segment'] = df_full['segment'].astype('category')
df_full['short_term_deposit'] = df_full['short_term_deposit'].astype('int32')
df_full['loans'] = df_full['loans'].astype('int32')
df_full['mortgage'] = df_full['mortgage'].astype('int32')
df_full['funds'] = df_full['funds'].astype('int32')
df_full['securities'] = df_full['securities'].astype('int32')
df_full['long_term_deposit'] = df_full['long_term_deposit'].astype('int32')
df_full['credit_card'] = df_full['credit_card'].astype('int32')
# df_full['payroll'] = df_full['payroll'].astype('int32')
# df_full['pension_plan'] = df_full['pension_plan'].astype('int32')
df_full['payroll_account'] = df_full['payroll_account'].astype('int32')
df_full['emc_account'] = df_full['emc_account'].astype('int32')
df_full['debit_card'] = df_full['debit_card'].astype('int32')
df_full['em_account_p'] = df_full['em_account_p'].astype('int32')
df_full['em_acount'] = df_full['em_acount'].astype('int32')
df_full["country_id"] = df_full["country_id"].astype('category')
df_full["region_code"] = df_full["region_code"].astype('category')
df_full["gender"] =  df_full["gender"].astype('category')
df_full["age"] = df_full["age"].astype('int32')

KeyError: "['Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0'] not found in axis"

## Análisis Preliminar

In [252]:
def build_my_info_table(df):
    # Seleccionar las columnas numéricas del DataFrame
    numerical_columns = df.select_dtypes(include=[np.number])
    
    # Crear un DataFrame con información detallada sobre las columnas
    df_info = pd.DataFrame({
        # Nombre de la columna
        'column':   [col             for col in df.columns],
        # Tipo de datos de la columna
        'dtype':    [df[col].dtype   for col in df.columns],
        # Número de valores no nulos en la columna
        'count':    [df[col].count() for col in df.columns],
        
        # Estadísticas descriptivas para columnas numéricas
        'mean':     [round(df[col].mean(), 1)         if col in numerical_columns else '' for col in df.columns],
        'std':      [round(df[col].std(), 1)          if col in numerical_columns else '' for col in df.columns],
        'min':      [round(df[col].min(), 1)          if col in numerical_columns else '' for col in df.columns],
        '25%':      [round(df[col].quantile(0.25), 1) if col in numerical_columns else '' for col in df.columns],
        '50%':      [round(df[col].median(), 1)       if col in numerical_columns else '' for col in df.columns],
        '75%':      [round(df[col].quantile(0.75), 1) if col in numerical_columns else '' for col in df.columns],
        'max':      [round(df[col].max(), 1)          if col in numerical_columns else '' for col in df.columns],
        
        # Número de valores únicos en la columna
        'nunique':  [df[col].nunique()                                          for col in df.columns],
        
        # Lista de valores únicos si el número de valores únicos es menor que 30, de lo contrario, cadena vacía
        'unique':   [list(df[col].unique()) if df[col].nunique() < 10 else ''   for col in df.columns],
        
        # Porcentaje de cardinalidad (número de valores únicos)
        'cardinality %': [round(df[col].nunique() / df.shape[0] * 100, 1)      for col in df.columns],
        
        # Moda (valor más frecuente) en la columna
        'mode':     [df[col].mode()[0]                                          for col in df.columns],
        
        # Porcentaje de ocurrencias de la moda en la columna
        'mode %':   [round(df[col].value_counts().max() * 100 / df.shape[0], 1) for col in df.columns],
        
        # Número de valores nulos en la columna
        'null_count': [df[col].isnull().sum() for col in df.columns],
        
        # Porcentaje de valores nulos en la columna
        'null %':   [round(df[col].isnull().mean() * 100, 1)                    for col in df.columns],
    })
    return df_info

In [253]:
# Crear un nuevo DataFrame df_soc con las columnas específicas
columnas_a_seleccionar = ['pk_cid', 'pk_partition', 'age', 'country_id', 'deceased', 'gender', 'region_code', 'salary']
df_soc = df_full[columnas_a_seleccionar]

df_soc.head()

,pk_cid,pk_partition,age,country_id,deceased,gender,region_code,salary
1479563,15891,2018-07-28,59,ES,N,H,28.000,NaN
2168122,15891,2018-08-28,59,ES,N,H,28.000,NaN
2962973,16063,2018-11-28,62,ES,N,H,28.000,NaN
3628236,16063,2018-12-28,62,ES,N,H,28.000,NaN
4028169,16063,2019-01-28,62,ES,N,H,28.000,NaN


In [254]:
df_soc.isna().sum()

pk_cid                0
pk_partition          0
age                   0
country_id            0
deceased              0
gender               25
region_code        2264
salary          1512103
dtype: int64

### Tabla de Información General

In [255]:
# Crear tabla de info.
df_soc_info = build_my_info_table(df_soc)

# Aplicar el estilo de gradiente de color.
styled_soc_df = df_soc_info.style.background_gradient(cmap='Reds', axis=0)

# Agregar estilos para hacer la tabla scrolleable y fijar los encabezados.
styled_soc_df = styled_soc_df.set_table_styles(
    [{
        'selector': 'thead th',
        'props': [('position', 'sticky'), ('top', '0'), ('background-color', 'black'), ('z-index', '1')]
    }, {
        'selector': 'tbody',
        'props': [('display', 'block'), ('max-height', '400px'), ('overflow-y', 'scroll'), ('overflow-x', 'auto')]
    }, {
        'selector': 'thead, tbody tr',
        'props': [('display', 'table'), ('width', '100%'), ('table-layout', 'fixed')]
    }, {
        'selector': 'th, td',
        'props': [('word-wrap', 'break-word'), ('white-space', 'pre-wrap'), ('max-width', '150px')]
    }]
)

# Mostrar la tabla.
styled_soc_df

,column,dtype,count,mean,std,min,25%,50%,75%,max,nunique,unique,cardinality %,mode,mode %,null_count,null %
0,pk_cid,int64,5962924,1234929.800000,162302.000000,15891,1112532.000000,1231097.000000,1352339.000000,1553689,456373,,7.700000,17457,0.000000,0,0.000000
1,pk_partition,datetime64[ns],5962924,,,,,,,,17,,0.000000,2019-05-28 00:00:00,7.400000,0,0.000000
2,age,int32,5962924,29.800000,12.000000,2,22.000000,25.000000,34.000000,105,104,,0.000000,22,12.200000,0,0.000000
3,country_id,category,5962924,,,,,,,,41,,0.000000,ES,100.000000,0,0.000000
4,deceased,object,5962924,,,,,,,,2,"['N', 'S']",0.000000,N,100.000000,0,0.000000
5,gender,category,5962899,,,,,,,,2,"['H', 'V', nan]",0.000000,H,51.800000,25,0.000000
6,region_code,category,5960660,,,,,,,,52,,0.000000,28.000000,19.900000,2264,0.000000
7,salary,float64,4450821,115816.700000,199551.900000,1202.700000,61500.600000,88654.600000,131669.900000,28894395.500000,258629,,4.300000,451931.220000,0.000000,1512103,25.400000


Haciendo un análisis preliminar de los resultados obtenidos en la tabla, podemos extraer laas siguientes conclusiones:

 - Se observan valores nulos en las variables de género, region_code & salary, estudiaremos como imputarlas más adelante.
 - Se observa una vairabilidad aceptable en todas las categoráis, a excepción de country (donde predomina España en casi un 100%) y deceased (practicamente ningún cliente fallecido).

## Mes pk_partition

In [256]:
# 'pk_partitions' formato de fecha
df_full['pk_partition'] = pd.to_datetime(df_full['pk_partition'], errors='coerce')

# Extraer el mes de la columna 'pk_partitions' y crear una nueva columna 'mes'
df_full['mes_partition'] = df_full['pk_partition'].dt.month

# Extraer también el nombre del mes:
df_full['mes_nombre_partition'] = df_full['pk_partition'].dt.strftime('%B')

Usaremos groupby() para obtener la última partición (pk_partition) y tener una visión actual de la empresa

In [257]:
# Filtrar el DataFrame para obtener solo los datos de la última ingesta
latest_date = '2019-05-28'
df_last_partition = df_full[df_full['pk_partition'] == latest_date]

## AGE

In [258]:
# Contar la distribución de valores en 'age' para la última partición
age_data = df_last_partition['age'].value_counts().reset_index()
age_data.columns = ['age', 'count']

# Filtrar las 15 edades más repetidas
top_15_ages = age_data.nlargest(15, 'count')

# Calcular el porcentaje de cada valor
total_count = top_15_ages['count'].sum()
top_15_ages['percentage'] = (top_15_ages['count'] / total_count) * 100

# Colores personalizados
custom_colors = ['#d56a61', '#eda476']  # Colores aproximados del gráfico original

# Crear el gráfico interactivo de barras
fig = px.bar(top_15_ages, 
             x='age', 
             y='count', 
             text='percentage',  # Mostrar el porcentaje en las etiquetas de las barras
             labels={'count':'Número de clientes', 'age':'Edad'}, 
             title='Top 15 Edades más comunes de clientes (última partición)',
             color_discrete_sequence=custom_colors)  # Usar la paleta de colores personalizada

# Personalizar el formato del texto de porcentaje
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

# Mostrar el gráfico interactivo
fig.show()

In [259]:
# Contar cuántos valores únicos de 'age' tiene cada cliente
estado_por_cliente_age = df_full.groupby('pk_cid')['age'].nunique()

# Identificar clientes con edades mixtas (múltiples valores en 'age')
clientes_mixtos_age = estado_por_cliente_age[estado_por_cliente_age > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_age = df_full[~df_full['pk_cid'].isin(clientes_mixtos_age)]

# Identificar clientes que tienen siempre la misma edad
clientes_misma_edad = df_no_mixtos_age.groupby('pk_cid')['age'].nunique()
clientes_misma_edad = clientes_misma_edad[clientes_misma_edad == 1].index

# Identificar clientes con valores NaN en 'age' (excluyendo clientes mixtos)
clientes_con_nan_age = df_no_mixtos_age[df_no_mixtos_age['age'].isna()]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_misma_edad = len(clientes_misma_edad)
n_con_nan_age = len(clientes_con_nan_age)
n_mixtos_age = len(clientes_mixtos_age)

# Mostrar los resultados
print(f"Clientes que tienen siempre la misma edad (excluyendo mixtos): {n_misma_edad}")
print(f"Clientes con valores NaN en 'age' (excluyendo mixtos): {n_con_nan_age}")
print(f"Clientes con edades mixtas (diferentes valores de edad): {n_mixtos_age}")

Clientes que tienen siempre la misma edad (excluyendo mixtos): 114631
Clientes con valores NaN en 'age' (excluyendo mixtos): 0
Clientes con edades mixtas (diferentes valores de edad): 341742


Vamos a analizar la distribución de grupos de edades gráficamente

In [260]:
# Crear el DataFrame con las tres categorías de clientes
clientes_data = pd.DataFrame({
    'categoria': ['Misma edad', 'Edades mixtas', 'Con NaN'],
    'count': [n_misma_edad, n_mixtos_age, n_con_nan_age]
})

# Definir los colores manualmente para cada categoría
custom_colors = {
    'Misma edad': '#e49467', 
    'Edades mixtas': '#f7f7c6', 
    'Con NaN': '#90c6a6'
}

# Crear el gráfico interactivo de barras con plotly
fig = px.bar(clientes_data, 
             x='categoria', 
             y='count', 
             text='count',  # Mostrar el número de casos en las etiquetas de las barras
             labels={'count':'Número de clientes', 'categoria':'Categoría de cliente'}, 
             title='Distribución de clientes según la categoría de edad',
             color='categoria',  # Añadir color basado en la categoría
             color_discrete_map=custom_colors)

# Personalizar el formato del texto para mostrar los números enteros
fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

# Mostrar el gráfico interactivo
fig.show()

In [261]:
n_total = n_misma_edad + n_con_nan_age + n_mixtos_age
n_total

456373

In [262]:
df_misma_edad = df_full[df_full['pk_cid'].isin(clientes_misma_edad)]
df_edad_mixta = df_full[df_full['pk_cid'].isin(clientes_mixtos_age)]

In [263]:
df_misma_edad["age"].describe()

count   1,200,493.000
mean           29.994
std            12.052
min             2.000
25%            22.000
50%            25.000
75%            35.000
max           102.000
Name: age, dtype: float64

In [264]:
df_edad_mixta["age"].describe()

count   4,762,431.000
mean           29.702
std            11.962
min             2.000
25%            22.000
50%            24.000
75%            34.000
max           105.000
Name: age, dtype: float64

Vamos a buscar los clientes con edades anómalas, y que podrían ser objetos de una modificación. En este caso vamos a ver primero aquellos que se mantienen siempre con la misma edad, y además son menores

### Edad Constante

1. Identificar clientes con productos activos
Primero, filtramos el DataFrame clientes_misma_edad_menores para obtener los clientes que tienen algún producto activo (donde el valor es 1).

In [265]:
# Filtrar clientes que siempre tienen la misma edad y esa edad es menor a 18 años
clientes_misma_edad_menores = df_misma_edad[df_misma_edad['age'] < 18]

# Mostrar los resultados
clientes_misma_edad_menores

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition
5520651,491473,2019-05-28,2019-04-27,KHM,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,V,2,N,NaN,5,May
5081423,541741,2019-04-28,2019-04-07,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,V,2,N,NaN,4,April
5520587,541741,2019-05-28,2019-04-07,KHK,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,V,2,N,NaN,5,May
1476977,617264,2018-07-28,2018-04-09,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,13,N,"286,573.440",7,July
1843524,617264,2018-08-28,2018-04-09,KFC,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,13,N,"286,573.440",8,August
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5679711,1552805,2019-05-28,2019-05-26,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,V,2,N,NaN,5,May
5679415,1552858,2019-05-28,2019-05-26,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,38.000,V,11,N,NaN,5,May
5680043,1553226,2019-05-28,2019-05-30,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,41.000,H,9,N,NaN,5,May
5680169,1553282,2019-05-28,2019-05-30,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,43.000,H,4,N,NaN,5,May


In [266]:
# Lista de productos a analizar
productos = [
    'short_term_deposit', 'loans', 'mortgage', 'funds', 'securities',
    'long_term_deposit', 'em_account_pp', 'credit_card', 'payroll', 
    'pension_plan', 'payroll_account', 'emc_account', 'debit_card', 
    'em_account_p', 'em_acount'
]

# Contar el número de casos donde cada producto ha sido contratado (valor = 1)
conteo_productos = clientes_misma_edad_menores[productos].sum()

# Mostrar el resultado
conteo_productos

short_term_deposit   0.000
loans                0.000
mortgage             0.000
funds                0.000
securities           0.000
long_term_deposit    0.000
em_account_pp        0.000
credit_card          0.000
payroll              2.000
pension_plan         9.000
payroll_account      0.000
emc_account          0.000
debit_card           0.000
em_account_p         0.000
em_acount            3.000
dtype: float64

In [267]:
pd.set_option('display.max_columns', None)

# Filtrar los clientes que tienen productos activos
clientes_con_productos_activos = clientes_misma_edad_menores[clientes_misma_edad_menores[productos].sum(axis=1) > 0]

# Mostrar el resultado
clientes_con_productos_activos

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition
5835610,1005293,2019-05-28,2015-02-16,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,1.000,0,0,0,0,0,ES,28.000,V,6,N,"81,032.520",5,May
5840803,1021968,2019-05-28,2015-07-04,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,1.000,0,0,0,0,0,ES,28.000,H,13,N,"318,767.100",5,May
5879905,1244964,2019-05-28,2017-02-24,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,1.000,0,0,0,0,0,ES,28.000,V,5,N,NaN,5,May
107719,1256153,2018-01-28,2017-05-05,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,1.000,0,0,0,0,0,ES,11.000,H,12,N,"156,534.360",1,January
1687485,1256153,2018-07-28,2017-05-05,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,1.000,0,0,0,0,0,ES,11.000,H,12,N,"156,534.360",7,July
4071542,1256153,2019-01-28,2017-05-05,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,1.000,0,0,0,0,0,ES,11.000,H,12,N,"156,534.360",1,January
5551912,1345478,2019-05-28,2017-10-31,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,1.000,0,0,0,0,0,ES,28.000,V,4,N,"118,693.500",5,May
1815226,1408784,2018-07-28,2018-07-20,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,22.000,V,17,N,"56,730.990",7,July
1925092,1408784,2018-08-28,2018-07-20,KHQ,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,22.000,V,17,N,"56,730.990",8,August
2458864,1408784,2018-09-28,2018-07-20,KHQ,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,22.000,V,17,N,"56,730.990",9,September


2. Repetir el proceso para los active_customer

In [268]:
# Filtrar los clientes menores de 18 años que también son active_customer
clientes_menores_active = clientes_misma_edad_menores[clientes_misma_edad_menores['active_customer'] == 1]

# Contar cuántos clientes cumplen con ambas condiciones
cantidad_menores_active = clientes_menores_active.shape[0]

# Mostrar la cantidad
print(f"Cantidad de clientes menores de 18 años que son active_customer: {cantidad_menores_active}")

# Mostrar la lista de estos clientes
clientes_menores_active

Cantidad de clientes menores de 18 años que son active_customer: 6053


,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition
1476977,617264,2018-07-28,2018-04-09,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,13,N,"286,573.440",7,July
83238,970787,2018-01-28,2015-09-05,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,14,N,"234,170.370",1,January
395797,970787,2018-02-28,2015-09-05,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,14,N,"234,170.370",2,February
635515,970787,2018-03-28,2015-09-05,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,14,N,"234,170.370",3,March
886240,970787,2018-04-28,2015-09-05,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,14,N,"234,170.370",4,April
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5681102,1552066,2019-05-28,2019-05-24,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,29.000,H,8,N,NaN,5,May
5681028,1552168,2019-05-28,2019-05-24,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,V,11,N,NaN,5,May
5680695,1552311,2019-05-28,2019-05-24,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,38.000,V,13,N,NaN,5,May
5680884,1552422,2019-05-28,2019-05-26,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,39.000,V,2,N,NaN,5,May


### Edad Mixta

En este apartado vamos a corregir errores de tipidficación en la variable Edad. Para ello, vamos a identificar en primer lugar casos en los que la edad tenga saltos mayores de 3 años (dado que el rango máximo de variación para un cliente debería ser este, pues la serie se compone de 3 años), y por otro lado, vamos a identificar aquellos casos en los que se produzca una variación negativa 

In [269]:
# Asegúrate de que df_edad_mixta esté ordenado por pk_cid y un índice de tiempo o la posición de las filas.
df_edad_mixta = df_edad_mixta.sort_values(by=['pk_cid', 'pk_partition'])  # Asumiendo que pk_partition representa el tiempo.

# Calcular la diferencia entre la primera y la última edad para cada usuario
df_edad_mixta['dif_edad'] = df_edad_mixta.groupby('pk_cid')['age'].transform(lambda x: x.iloc[-1] - x.iloc[0])

# Mostrar el resultado
df_edad_mixta[['pk_cid', 'age', 'dif_edad']].drop_duplicates()

,pk_cid,age,dif_edad
2171490,16502,57,1
2943633,16502,58,1
237929,17457,53,1
5134250,17457,54,1
1479558,17590,54,1
...,...,...,...
5683006,1548180,38,-1
5100662,1548181,34,1
5683005,1548181,35,1
5100664,1548186,42,-1


#### Edad Mixta. Variación > 3 años

In [270]:
df_edad_mixta_dif_mas3 = df_edad_mixta[df_edad_mixta['dif_edad'] > 3]

df_edad_mixta_dif_mas3["dif_edad"].describe()

count   1,093.000
mean       25.080
std        14.549
min         5.000
25%        14.000
50%        21.000
75%        31.000
max        67.000
Name: dif_edad, dtype: float64

In [271]:
df_edad_mixta[df_edad_mixta['dif_edad'] > 60]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition,dif_edad
2169124,539069,2018-09-28,2018-09-21,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,46.000,V,2,N,NaN,9,September,67
2945655,539069,2018-10-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,46.000,V,2,N,NaN,10,October,67
2955191,539069,2018-11-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,ES,46.000,V,69,N,NaN,11,November,67
3630152,539069,2018-12-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,46.000,V,69,N,NaN,12,December,67
4102655,539069,2019-01-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,ES,46.000,V,69,N,NaN,1,January,67
4479766,539069,2019-02-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,ES,46.000,V,69,N,NaN,2,February,67
4886596,539069,2019-03-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,46.000,V,69,N,NaN,3,March,67
5080982,539069,2019-04-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,46.000,V,69,N,NaN,4,April,67
5520571,539069,2019-05-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,ES,46.000,V,69,N,NaN,5,May,67
1324522,1308540,2018-06-28,2017-09-16,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,16.000,V,21,N,"116,896.110",6,June,61


In [272]:
df_edad_mixta[(df_edad_mixta['dif_edad'] > 3) & (df_edad_mixta['dif_edad'] < 6)]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition,dif_edad
1813962,1405875,2018-07-28,2018-07-13,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,40.000,V,20,N,"116,051.580",7,July,5
1929750,1405875,2018-08-28,2018-07-13,KHM,1,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,40.000,V,24,N,"116,051.580",8,August,5
2466761,1405875,2018-09-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,40.000,V,24,N,"116,051.580",9,September,5
2733258,1405875,2018-10-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,40.000,V,24,N,"116,051.580",10,October,5
3177827,1405875,2018-11-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,40.000,V,25,N,"116,051.580",11,November,5
3776945,1405875,2018-12-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,40.000,V,25,N,"116,051.580",12,December,5
3959819,1405875,2019-01-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,40.000,V,25,N,"116,051.580",1,January,5
4415986,1405875,2019-02-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,40.000,V,25,N,"116,051.580",2,February,5
4676045,1405875,2019-03-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,40.000,V,25,N,"116,051.580",3,March,5
5315811,1405875,2019-04-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,40.000,V,25,N,"116,051.580",4,April,5


En este caso, vamos a sustituir todos los valores de Age por el último valor conocido, ya que, a futuro lo más coherente será crear categorías de edad en las que se agrupen distintas décadas, por tanto, las variaciones internas de cada cliente van a ser practicamente irrelevantes (si un cliente pasa de 30-33 no es diferenciador), y además, se sobreentiende que los errores de tipificación se corrigen con el tiempo, por tanto, cuanto más reciente sea una entrada, más probable es que sea correcta.

In [273]:
# Paso 1: Filtrar los individuos que cumplen la condición dif_edad > 3
df_edad_mixta_dif_mas3 = df_edad_mixta[df_edad_mixta['dif_edad'] > 3]

# Paso 2: Obtener la última edad registrada (en la última pk_partition) para cada individuo
ultima_edad_por_individuo = df_edad_mixta_dif_mas3.groupby('pk_cid').apply(lambda x: x.loc[x['pk_partition'].idxmax(), 'age'])

# Paso 3: Actualizar la edad en df_full para los individuos que cumplen la condición
def actualizar_edad(row):
    if row['pk_cid'] in ultima_edad_por_individuo.index:
        return ultima_edad_por_individuo[row['pk_cid']]
    else:
        return row['age']

# Aplicar la función para actualizar la edad en df_full
df_full['age'] = df_full.apply(actualizar_edad, axis=1)

# Mostrar el resultado
df_full.head()

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition
1479563,15891,2018-07-28,2018-07-28,KAT,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,28.000,H,59,N,NaN,7,July
2168122,15891,2018-08-28,2018-07-28,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,59,N,NaN,8,August
2962973,16063,2018-11-28,2018-11-19,KAT,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,62,N,NaN,11,November
3628236,16063,2018-12-28,2018-11-19,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,62,N,NaN,12,December
4028169,16063,2019-01-28,2018-11-19,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,62,N,NaN,1,January


#### Edad Mixta. Variación Negativa

In [274]:
# Filtrar los casos donde la diferencia de edad es menor que 0
df_dif_edad_neg = df_edad_mixta[df_edad_mixta['dif_edad'] < 0]

df_dif_edad_neg["dif_edad"].describe()

count   94,401.000
mean        -1.011
std          0.423
min        -40.000
25%         -1.000
50%         -1.000
75%         -1.000
max         -1.000
Name: dif_edad, dtype: float64

In [275]:
# Visualizar los casos donde la diferencia de edad es menor que 0
df_dif_edad_neg

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition,dif_edad
2943726,46732,2018-10-28,2018-10-08,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,46.000,V,71,N,NaN,10,October,-1
2977216,46732,2018-11-28,2018-10-08,KFC,1,02 - PARTICULARES,0,0,0,0,0,1,0,0,0.000,0.000,0,0,0,0,1,ES,46.000,V,70,N,NaN,11,November,-1
3628313,46732,2018-12-28,2018-10-08,KFC,1,02 - PARTICULARES,0,0,0,0,0,1,0,0,0.000,0.000,0,0,0,0,1,ES,46.000,V,70,N,NaN,12,December,-1
3839343,46732,2019-01-28,2018-10-08,KFC,1,02 - PARTICULARES,0,0,0,0,0,1,0,0,0.000,0.000,0,0,0,0,1,ES,46.000,V,70,N,NaN,1,January,-1
4480561,46732,2019-02-28,2018-10-08,KFC,1,02 - PARTICULARES,0,0,0,0,0,1,0,0,0.000,0.000,0,0,0,0,1,ES,46.000,V,70,N,NaN,2,February,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5682969,1548156,2019-05-28,2019-04-29,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,50.000,V,62,N,NaN,5,May,-1
5100661,1548180,2019-04-28,2019-04-29,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,H,39,N,NaN,4,April,-1
5683006,1548180,2019-05-28,2019-04-29,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,28.000,H,38,N,NaN,5,May,-1
5100664,1548186,2019-04-28,2019-04-29,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,3.000,V,42,N,NaN,4,April,-1


In [276]:
df_dif_edad_neg[df_dif_edad_neg['dif_edad'] < -1]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition,dif_edad
3033723,308667,2018-11-28,2018-11-16,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,50.000,V,65,N,NaN,11,November,-9
3627875,308667,2018-12-28,2018-11-16,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,50.000,V,65,N,NaN,12,December,-9
4148494,308667,2019-01-28,2018-11-16,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,50.000,V,55,N,NaN,1,January,-9
4481102,308667,2019-02-28,2018-11-16,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,ES,50.000,V,55,N,NaN,2,February,-9
4884600,308667,2019-03-28,2018-11-16,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,ES,50.000,V,55,N,NaN,3,March,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5698512,1526523,2019-05-28,2019-01-22,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,ES,28.000,H,34,N,NaN,5,May,-10
4394326,1530367,2019-02-28,2019-02-05,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,32.000,V,86,N,NaN,2,February,-40
4720580,1530367,2019-03-28,2019-02-05,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,1.000,1.000,1,0,1,0,0,ES,32.000,V,86,N,NaN,3,March,-40
5084070,1530367,2019-04-28,2019-02-05,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,1.000,1.000,1,0,1,0,0,ES,32.000,V,46,N,NaN,4,April,-40


De igual forma vamos a sustituir todas las edades de los casos con variaciones negativas por la edad que figure en la última entrada de cada usuario

In [277]:
# Paso 1: Obtener la última edad registrada (en la última pk_partition) para cada individuo
ultima_edad_por_individuo_neg = df_dif_edad_neg.groupby('pk_cid').apply(lambda x: x.loc[x['pk_partition'].idxmax(), 'age'])

# Paso 2: Actualizar la edad en df_full para los individuos que cumplen la condición
def actualizar_edad_neg(row):
    if row['pk_cid'] in ultima_edad_por_individuo_neg.index:
        return ultima_edad_por_individuo_neg[row['pk_cid']]
    else:
        return row['age']

# Aplicar la función para actualizar la edad en df_full
df_full['age'] = df_full.apply(actualizar_edad_neg, axis=1)

## COUNTRY ID

In [278]:
# Contar la distribución de valores en 'country_id' para la última partición
country_data = df_last_partition['country_id'].value_counts().reset_index()
country_data.columns = ['country_id', 'count']

# Filtrar los 10 países más repetidos
top_10_countries = country_data.nlargest(10, 'count')

# Crear el gráfico interactivo de barras con plotly
custom_colors = ['#e49467', '#f7f7c6', '#90c6a6']  # Usar la paleta de colores anterior

fig = px.bar(top_10_countries, 
             x='country_id', 
             y='count', 
             text='count',  # Mostrar el número de casos en las etiquetas de las barras
             labels={'count':'Número de clientes', 'country_id':'País'}, 
             title='Top 10 Países más comunes de clientes (última partición)',
             color_discrete_sequence=custom_colors)

# Personalizar el formato del texto para mostrar los números enteros
fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

# Mostrar el gráfico interactivo
fig.show()

In [279]:
# Contar la distribución de valores en 'country_id' para la última partición
country_data = df_last_partition['country_id'].value_counts().reset_index()
country_data.columns = ['country_id', 'count']

# Excluir el país 'ES'
country_data = country_data[country_data['country_id'] != 'ES']

# Filtrar los 10 países más repetidos excluyendo 'ES'
top_10_countries = country_data.nlargest(10, 'count')

# Crear el gráfico interactivo de barras con plotly
fig = px.bar(top_10_countries, 
             x='country_id', 
             y='count', 
             text='count',  # Mostrar el número de casos en las etiquetas de las barras
             labels={'count':'Número de clientes', 'country_id':'País'}, 
             title='Top 10 Países más comunes de clientes (sin "ES")',
             color='count',  # Usar la columna 'count' para variar el color de las barras
             color_continuous_scale=custom_colors)  # Usar la paleta de colores personalizada

# Personalizar el formato del texto para mostrar los números enteros
fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

# Mostrar el gráfico interactivo
fig.show()

In [280]:
# Contar cuántos valores únicos de 'country_id' tiene cada cliente
estado_por_cliente_country = df_full.groupby('pk_cid')['country_id'].nunique()

# Identificar clientes con 'country_id' mixtos (múltiples valores de 'country_id')
clientes_mixtos_country = estado_por_cliente_country[estado_por_cliente_country > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_country = df_full[~df_full['pk_cid'].isin(clientes_mixtos_country)]

# Identificar clientes que tienen siempre el mismo 'country_id'
clientes_mismo_country = df_no_mixtos_country.groupby('pk_cid')['country_id'].nunique()
clientes_mismo_country = clientes_mismo_country[clientes_mismo_country == 1].index

# Identificar clientes con valores NaN en 'country_id' (excluyendo clientes mixtos)
clientes_con_nan_country = df_no_mixtos_country[df_no_mixtos_country['country_id'].isna()]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_mismo_country = len(clientes_mismo_country)
n_con_nan_country = len(clientes_con_nan_country)
n_mixtos_country = len(clientes_mixtos_country)

# Mostrar los resultados
print(f"Clientes que tienen siempre el mismo 'country_id' (excluyendo mixtos): {n_mismo_country}")
print(f"Clientes con valores NaN en 'country_id' (excluyendo mixtos): {n_con_nan_country}")
print(f"Clientes con 'country_id' mixtos (diferentes países): {n_mixtos_country}")

Clientes que tienen siempre el mismo 'country_id' (excluyendo mixtos): 456348
Clientes con valores NaN en 'country_id' (excluyendo mixtos): 0
Clientes con 'country_id' mixtos (diferentes países): 25


In [281]:
# Crear un DataFrame con los conteos de cada categoría
data = {
    'Categoría': ['Clientes con el mismo country_id', 
                  'Clientes con valores NaN', 
                  'Clientes con country_id mixtos'],
    'Número de clientes': [n_mismo_country, 
                           n_con_nan_country, 
                           n_mixtos_country]
}

df_categorias = pd.DataFrame(data)

# Definir los colores personalizados para cada categoría
custom_colors = {
    'Clientes con el mismo country_id': '#e49467', 
    'Clientes con valores NaN': '#f7f7c6', 
    'Clientes con country_id mixtos': '#90c6a6'
}

# Crear el gráfico interactivo de barras con plotly
fig = px.bar(df_categorias, 
             x='Categoría', 
             y='Número de clientes', 
             text='Número de clientes',
             labels={'Número de clientes':'Número de clientes', 'Categoría':'Categoría'}, 
             title='Distribución de Clientes por Categoría de country_id',
             color='Categoría',
             color_discrete_map=custom_colors)

# Personalizar el formato del texto para mostrar los números enteros
fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

# Mostrar el gráfico interactivo
fig.show()

Los clientes con country id distintos como que antes vivian en un pais y ahora se han mudado para otro, no sé como tratar esto, si mantener el ultimo o el primero. (Adrián) Yo no los cambiaría, la variable indica país de residencia, por tanto, puede ser variable, y son muy pocos casos, no creo que supongan una distorsión de resultados

In [282]:
# Lista de valores únicos en la columna 'country_id'
df_full['country_id'].value_counts()

country_id
ES    5960672
GB        441
FR        225
DE        199
US        195
CH        194
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
DO         17
SN         17
SA         17
RU         17
QA         17
NO         17
CI         17
MR         17
LU         17
CM         17
GA         17
CO         17
ET         17
GT         17
SE         16
JM         11
PT         11
DJ         11
RO          9
HU          8
DZ          7
PE          4
Name: count, dtype: int64

In [283]:
# Filtrar el DataFrame para obtener las filas donde el país es mixto
df_full[df_full['pk_cid'].isin(clientes_mixtos_country)]


,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition
1843228,670953,2018-08-28,2018-08-11,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,8.000,V,39,N,"111,759.690",8,August
2268552,670953,2018-09-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,8.000,V,39,N,"111,759.690",9,September
2946158,670953,2018-10-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,8.000,V,39,N,"111,759.690",10,October
2954597,670953,2018-11-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,8.000,V,39,N,"111,759.690",11,November
3628803,670953,2018-12-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,NaN,V,39,N,"111,759.690",12,December
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4730692,1519831,2019-03-28,2018-12-29,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,GB,NaN,H,26,N,NaN,3,March
5121965,1519831,2019-04-28,2018-12-29,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,GB,NaN,H,26,N,NaN,4,April
5686361,1519831,2019-05-28,2018-12-29,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,GB,NaN,H,26,N,NaN,5,May
5096824,1542679,2019-04-28,2019-03-31,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,7.000,V,64,N,NaN,4,April


### Country_ID Mixto

En este caso también vamos a modificar los casos de clientes con un country_id mixto en el DataFrame df_full, y asignarles el último país registrado para su pk_cid, ya que al tratarse solo de 25 clientes, todo parece indicar que se trata de errores en la toma de datos. Además, suponiendo que estos cambios de residencia fueran reales, entonces el salario debería sufrir modificaciones, situación que no se observa en ninguno de los casos. Por tanto, nos vamos a quedar con la información más actualizada de cada individuo

In [284]:
# Obtener el último valor de 'country_id' para cada cliente mixto
ultimo_country_por_cliente = df_full[df_full['pk_cid'].isin(clientes_mixtos_country)].groupby('pk_cid').apply(lambda x: x.loc[x['pk_partition'].idxmax(), 'country_id'])

In [285]:
# Función para actualizar el country_id según la última entrada
def actualizar_country_mixto(row):
    if row['pk_cid'] in ultimo_country_por_cliente.index:
        return ultimo_country_por_cliente[row['pk_cid']]
    else:
        return row['country_id']

# Aplicar la función para actualizar el country_id en df_full
df_full['country_id'] = df_full.apply(actualizar_country_mixto, axis=1)

# Verificar los cambios en los clientes mixtos
df_mixtos_actualizados_country = df_full[df_full['pk_cid'].isin(clientes_mixtos_country)]

# Mostrar algunos resultados para verificar
df_mixtos_actualizados_country.head(10)

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition
1843228,670953,2018-08-28,2018-08-11,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,8.000,V,39,N,"111,759.690",8,August
2268552,670953,2018-09-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,8.000,V,39,N,"111,759.690",9,September
2946158,670953,2018-10-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,8.000,V,39,N,"111,759.690",10,October
2954597,670953,2018-11-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,8.000,V,39,N,"111,759.690",11,November
3628803,670953,2018-12-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,NaN,V,39,N,"111,759.690",12,December
4103151,670953,2019-01-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,NaN,V,40,N,"111,759.690",1,January
4479705,670953,2019-02-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,NaN,V,40,N,"111,759.690",2,February
4887293,670953,2019-03-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,NaN,V,40,N,"111,759.690",3,March
5080396,670953,2019-04-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,NaN,V,40,N,"111,759.690",4,April
5520009,670953,2019-05-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,NaN,V,40,N,"111,759.690",5,May


## DECEASED

In [286]:
# Contar cuántos valores únicos de 'deceased' tiene cada cliente
estado_por_cliente_deceased = df_full.groupby('pk_cid')['deceased'].nunique()

# Identificar clientes con estados mixtos (N y S o con NaN incluido)
clientes_mixtos_deceased = estado_por_cliente_deceased[estado_por_cliente_deceased > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_deceased = df_full[~df_full['pk_cid'].isin(clientes_mixtos_deceased)]

# Identificar clientes que son siempre 'N'
clientes_siempre_n = df_no_mixtos_deceased[df_no_mixtos_deceased['deceased'] == 'N']['pk_cid'].unique()

# Identificar clientes que son siempre 'S'
clientes_siempre_s = df_no_mixtos_deceased[df_no_mixtos_deceased['deceased'] == 'S']['pk_cid'].unique()

# Identificar clientes con valores NaN en 'deceased' (excluyendo clientes mixtos)
clientes_con_nan_deceased = df_no_mixtos_deceased[df_no_mixtos_deceased['deceased'].isna()]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_siempre_n = len(clientes_siempre_n)
n_siempre_s = len(clientes_siempre_s)
n_con_nan_deceased = len(clientes_con_nan_deceased)
n_mixtos_deceased = len(clientes_mixtos_deceased)

# Mostrar los resultados
print(f"Clientes que son siempre 'N' en 'deceased' (excluyendo mixtos): {n_siempre_n}")
print(f"Clientes que son siempre 'S' en 'deceased' (excluyendo mixtos): {n_siempre_s}")
print(f"Clientes con valores NaN en 'deceased' (excluyendo mixtos): {n_con_nan_deceased}")
print(f"Clientes con estados mixtos en 'deceased' (N, S o NaN): {n_mixtos_deceased}")

Clientes que son siempre 'N' en 'deceased' (excluyendo mixtos): 456244
Clientes que son siempre 'S' en 'deceased' (excluyendo mixtos): 55
Clientes con valores NaN en 'deceased' (excluyendo mixtos): 0
Clientes con estados mixtos en 'deceased' (N, S o NaN): 74


In [287]:
n_total = n_siempre_n + n_siempre_s + n_con_nan_deceased + n_mixtos_deceased
n_total

456373

hay 55 clientes muertos desde la primera ingesta de datos incluido el entry_date, serán errores o se tendrán que analizar mas a fondo así como los que cambian

### Deceased Constante

In [288]:
# Filtrar los clientes que son siempre 'S' en 'deceased'
df_siempre_s = df_full[df_full['pk_cid'].isin(clientes_siempre_s)]

# Ahora muestra la tabla completa para los clientes que son siempre 'S' en 'deceased'
df_siempre_s['age'].value_counts()

age
83     41
84     41
90     40
89     27
98     25
72     25
58     25
85     24
54     23
25     22
61     22
59     21
88     20
93     19
53     18
95     17
21     16
75     16
77     16
48     16
86     15
38     14
87     14
101    14
26     12
69     12
100    12
92     12
60     11
102    10
91     10
28     10
76     10
20      9
70      9
35      9
22      9
63      8
42      8
43      8
65      8
57      7
27      7
56      7
52      7
41      7
71      5
34      5
55      4
66      3
23      2
99      2
39      2
62      1
49      1
94      1
Name: count, dtype: int64

### Deceased Mixto

In [289]:
# Filtrar los clientes que tienen estados mixtos en 'deceased'
df_mixtos_deceased = df_full[df_full['pk_cid'].isin(clientes_mixtos_deceased)]

df_mixtos_deceased['age'].value_counts()

age
95     45
65     42
55     39
91     39
86     39
54     37
80     33
81     33
43     33
53     33
52     32
62     31
44     31
68     30
76     30
96     27
87     27
75     26
66     26
83     25
21     24
56     21
45     20
59     19
92     18
90     17
93     16
23     16
85     15
67     15
100    15
47     14
89     14
58     14
79     14
78     14
64     14
97     13
72     13
88     12
48     12
69     11
82     10
38      9
70      9
71      8
46      7
22      6
31      6
94      5
101     4
39      3
60      3
77      3
63      2
99      1
24      1
32      1
Name: count, dtype: int64

Como podemos comprobar, por un lado, la variable deceased tiene una variabilidad practicamente despreciable, por tanto, es lógico pensar que dicha variable no va a tener una capacidad explicativa suficiente.

No vamos a tener en cuenta la variable deceased para hacer modelos al no tener ninguna capacidad predictiva ni explicativa.

Además dada su alta correlación con otras variables como el grupo de edad, es prudente elimiarla en los modleos para no caer en problemas de multicolinealidad entre las variables explicativas

## GENDER

In [290]:
# Definir los colores personalizados para cada género
custom_colors = ['#e49467', '#f7f7c6']

# Contar la distribución de valores en 'gender'
gender_data = df_last_partition['gender'].value_counts().reset_index()
gender_data.columns = ['gender', 'count']

# Crear el gráfico interactivo de barras con plotly
fig = px.bar(gender_data, 
             x='gender', 
             y='count', 
             text='count',  # Mostrar el número de casos en las etiquetas de las barras
             labels={'count':'Número de clientes', 'gender':'Género'}, 
             title='Distribución de Clientes por Género',
             color='gender',
             color_discrete_sequence=custom_colors)

# Personalizar el formato del texto para mostrar los números enteros
fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

# Mostrar el gráfico interactivo
fig.show()

In [291]:
# se crea una categoria Unknown para los valores nulos
df_full["gender"] = df_full["gender"].cat.add_categories(["Unknown"]).fillna("Unknown")

df_full.groupby("gender")["pk_cid"].nunique().apply(lambda x: x/df_full["pk_cid"].nunique()*100)

gender
H         51.244
V         48.765
Unknown    0.000
Name: pk_cid, dtype: float64

In [292]:
df_full.groupby("gender")["pk_cid"].nunique()

gender
H          233862
V          222550
Unknown         2
Name: pk_cid, dtype: int64

In [293]:
df_full.groupby("gender")["pk_cid"].nunique().sum()
# se observa que hay 41 valores más que la cantidad de clientes únicos
# lo que representa que hay clientes con más de un género asignado

456414

### Análisis Géneros

In [294]:

# Contar cuántos valores únicos tiene cada cliente en la columna gender
estado_por_cliente_gender = df_full.groupby('pk_cid')['gender'].agg(['nunique', 'count'])

# Identificar clientes con géneros mixtos (H y F o Unknown)
clientes_mixtos_gender = estado_por_cliente_gender[estado_por_cliente_gender['nunique'] > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_gender = df_full[~df_full['pk_cid'].isin(clientes_mixtos_gender)]

# Identificar clientes que son siempre 'H'
clientes_siempre_H = df_no_mixtos_gender[df_no_mixtos_gender['gender'] == 'H']['pk_cid'].unique()

# Identificar clientes que son siempre 'V'
clientes_siempre_V = df_no_mixtos_gender[df_no_mixtos_gender['gender'] == 'V']['pk_cid'].unique()

# Identificar clientes "Unknown" en gender (excluyendo clientes mixtos)
clientes_siempre_Unknown = df_no_mixtos_gender[df_no_mixtos_gender['gender'] == "Unknown"]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_siempre_H = len(clientes_siempre_H)
n_siempre_V = len(clientes_siempre_V)
n_siempre_Unknown = len(clientes_siempre_Unknown)
n_mixtos_gender = len(clientes_mixtos_gender)

# Mostrar los resultados
print(f"Clientes que son siempre 'H' en 'gender' (excluyendo mixtos): {n_siempre_H}")
print(f"Clientes que son siempre 'V' en 'gender' (excluyendo mixtos): {n_siempre_V}")
print(f"Clientes que son todo 'Unknown' (excluyendo mixtos): {n_siempre_Unknown}")
print(f"Clientes con géneros mixtos (H y V o Unknown): {n_mixtos_gender}")
print(f"Total de clientes en el dataset: {n_siempre_H + n_siempre_V + n_siempre_Unknown + n_mixtos_gender}")


Clientes que son siempre 'H' en 'gender' (excluyendo mixtos): 233821
Clientes que son siempre 'V' en 'gender' (excluyendo mixtos): 222509
Clientes que son todo 'Unknown' (excluyendo mixtos): 2
Clientes con géneros mixtos (H y V o Unknown): 41
Total de clientes en el dataset: 456373


In [295]:
# Poroporciones
n_total = n_siempre_H + n_siempre_V + n_siempre_Unknown + n_mixtos_gender
p_siempre_H = n_siempre_H / n_total * 100
p_siempre_V = n_siempre_V / n_total * 100
p_siempre_Unknown = n_siempre_Unknown / n_total * 100
p_mixtos_gender = n_mixtos_gender / n_total * 100

# Mostrar los resultados
print(f"Clientes que son siempre 'H': {p_siempre_H:.2f}%")
print(f"Clientes que son siempre 'V': {p_siempre_V:.2f}%")
print(f"Clientes que son todo 'Unknown': {p_siempre_Unknown:.5f}%")
print(f"Clientes con géneros mixtos (H y V o Unknown): {p_mixtos_gender:.3f}%")

Clientes que son siempre 'H': 51.23%
Clientes que son siempre 'V': 48.76%
Clientes que son todo 'Unknown': 0.00044%
Clientes con géneros mixtos (H y V o Unknown): 0.009%


### Género unknown constante

In [296]:
# Clientes donde el valor de 'gender' es 'Unknown'
df_full[df_full['gender'] == "Unknown"]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition
2168807,216507,2018-08-28,2018-08-17,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,Unknown,72,N,"104,296.620",8,August
2172127,216507,2018-09-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,Unknown,72,N,"104,296.620",9,September
2943933,216507,2018-10-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,28.000,Unknown,72,N,"104,296.620",10,October
3033652,216507,2018-11-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,0,ES,28.000,Unknown,72,N,"104,296.620",11,November
3627544,216507,2018-12-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,0,ES,28.000,Unknown,72,N,"104,296.620",12,December
3839882,216507,2019-01-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,0,ES,28.000,Unknown,73,N,"104,296.620",1,January
4481238,216507,2019-02-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,0,ES,28.000,Unknown,72,N,"104,296.620",2,February
4875861,216507,2019-03-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,0,ES,28.000,Unknown,73,N,"104,296.620",3,March
5133231,216507,2019-04-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,0,ES,28.000,Unknown,73,N,"104,296.620",4,April
5523191,216507,2019-05-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,0,ES,28.000,Unknown,73,N,"104,296.620",5,May


In [297]:
# Reemplazar el valor 'Unknown' por 'H' en la columna 'gender'
df_full.loc[df_full['gender'] == "Unknown", 'gender'] = "H"

Vamos a imputar los Unknown y los vamos a cambiar por Hombre, ya que, ambos individuos superan los 70 años (sería extraño que se identiffiquen como género fluido y la variable H es la que presenta mayor proporción, por tanto lo imputaremos a esta)

### Género Unknown Mixto

In [298]:
# Aplicar el método ffill y bfill para imputar Unknown dentro de cada grupo
df_full['gender'] = df_full.groupby('pk_cid')['gender'].transform(lambda x: x.ffill().bfill())

# Verificar si hay todavía valores Unknown después de la imputación
unknown_after = df_full[df_full['gender'] == "Unknown"]

print(f"Número de valores 'Unknown' después de la imputación: {len(unknown_after)}")


Número de valores 'Unknown' después de la imputación: 0


### Géneros Mixtos

También vamos a modificar los casos en lso que el género es mixto, tomando el último valor registrado para cada individuo, ya que como hemos apuntado anteriormente, es lógico pensar que la última entrada de cada cliente es la más actualizada, y por tanto, la correcta

In [299]:
# Obtener el último valor de 'gender' para cada cliente mixto
ultimo_genero_por_cliente = df_full[df_full['pk_cid'].isin(clientes_mixtos_gender)].groupby('pk_cid').apply(lambda x: x.loc[x['pk_partition'].idxmax(), 'gender'])

In [300]:
# Función para actualizar el género según la última entrada
def actualizar_genero_mixto(row):
    if row['pk_cid'] in ultimo_genero_por_cliente.index:
        return ultimo_genero_por_cliente[row['pk_cid']]
    else:
        return row['gender']

# Aplicar la función para actualizar el género en df_full
df_full['gender'] = df_full.apply(actualizar_genero_mixto, axis=1)

# Verificar los cambios en los clientes mixtos
df_mixtos_actualizados = df_full[df_full['pk_cid'].isin(clientes_mixtos_gender)]

In [301]:
# Mostrar algunos resultados para verificar
df_mixtos_actualizados.head(20)

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition
81224,1012963,2018-01-28,2015-04-23,KHD,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,15.000,H,24,N,"132,481.590",1,January
403796,1012963,2018-02-28,2015-04-23,KHD,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,15.000,H,24,N,"132,481.590",2,February
640152,1012963,2018-03-28,2015-04-23,KHD,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,15.000,H,24,N,"132,481.590",3,March
894546,1012963,2018-04-28,2015-04-23,KHD,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,15.000,H,24,N,"132,481.590",4,April
1063018,1012963,2018-05-28,2015-04-23,KHD,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,15.000,H,24,N,"132,481.590",5,May
1315419,1012963,2018-06-28,2015-04-23,KHD,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,15.000,H,24,N,"132,481.590",6,June
1612841,1012963,2018-07-28,2015-04-23,KHD,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,15.000,H,24,N,"132,481.590",7,July
1832939,1012963,2018-08-28,2015-04-23,KHD,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,15.000,H,24,N,"132,481.590",8,August
2324883,1012963,2018-09-28,2015-04-23,KHD,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,15.000,H,24,N,"132,481.590",9,September
2660594,1012963,2018-10-28,2015-04-23,KHD,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,15.000,H,24,N,"132,481.590",10,October


## REGION_CODE


In [302]:
# valores nulos en region_code ponerlos como Unknown
df_full["region_code"] = df_full["region_code"].cat.add_categories(["Unknown"]).fillna("Unknown")

In [303]:
# Contar cuántos valores únicos de 'region_code' tiene cada cliente
estado_por_cliente_region = df_full.groupby('pk_cid')['region_code'].nunique()

# Identificar clientes con múltiples códigos de región (cambios de región)
clientes_mixtos_region = estado_por_cliente_region[estado_por_cliente_region > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_region = df_full[~df_full['pk_cid'].isin(clientes_mixtos_region)]

# Identificar clientes que tienen siempre el mismo código de región
clientes_mismo_region = df_no_mixtos_region.groupby('pk_cid')['region_code'].nunique()
clientes_mismo_region = clientes_mismo_region[clientes_mismo_region == 1].index

# Identificar clientes con valores NaN en 'region_code' (excluyendo clientes mixtos)
clientes_con_nan_region = df_no_mixtos_region[df_no_mixtos_region['region_code'].isna()]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_mismo_region = len(clientes_mismo_region)
n_con_nan_region = len(clientes_con_nan_region)
n_mixtos_region = len(clientes_mixtos_region)

# Mostrar los resultados
print(f"Clientes que tienen siempre el mismo código de región (excluyendo mixtos): {n_mismo_region}")
print(f"Clientes con valores NaN en 'region_code' (excluyendo mixtos): {n_con_nan_region}")
print(f"Clientes con códigos de región mixtos (diferentes códigos): {n_mixtos_region}")

Clientes que tienen siempre el mismo código de región (excluyendo mixtos): 454305
Clientes con valores NaN en 'region_code' (excluyendo mixtos): 0
Clientes con códigos de región mixtos (diferentes códigos): 2068


In [304]:
# se comprueba que la suma de los clientes en cada región es igual a la cantidad de clientes únicos
n_total = n_mismo_region + n_con_nan_region + n_mixtos_region
n_total 

456373

In [305]:
df_full[df_full["region_code"] == 'Unknown']

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition
240946,130419,2018-02-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,1,0.000,0.000,0,0,0,0,0,US,Unknown,V,53,N,NaN,2,February
483522,130419,2018-03-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,US,Unknown,V,53,N,NaN,3,March
728777,130419,2018-04-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,1,0.000,0.000,0,0,0,0,0,US,Unknown,V,53,N,NaN,4,April
1223095,130419,2018-05-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,US,Unknown,V,53,N,NaN,5,May
1479139,130419,2018-07-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,US,Unknown,V,53,N,NaN,7,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083649,1530982,2019-04-28,2019-02-09,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,GB,Unknown,V,33,N,NaN,4,April
5692590,1530982,2019-05-28,2019-02-09,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,1.000,1.000,1,0,0,0,0,GB,Unknown,V,33,N,NaN,5,May
5672371,1542679,2019-05-28,2019-03-31,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,US,Unknown,V,64,N,NaN,5,May
5097292,1544346,2019-04-28,2019-04-08,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,FR,Unknown,H,46,N,NaN,4,April


In [306]:
# Filtra el DataFrame según las condiciones especificadas
filtered_df = df_full[(df_full["region_code"] == 'Unknown') & (df_full["country_id"] == 'ES')]

filtered_df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition
4320,1054714,2018-01-28,2015-08-29,KAT,1,01 - TOP,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Unknown,V,40,N,NaN,1,January
480274,1054714,2018-02-28,2015-08-29,KAT,1,01 - TOP,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Unknown,V,40,N,NaN,2,February
720817,1054714,2018-03-28,2015-08-29,KAT,1,01 - TOP,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,Unknown,V,40,N,NaN,3,March
965232,1054714,2018-04-28,2015-08-29,KAT,1,01 - TOP,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,Unknown,V,40,N,NaN,4,April
980872,1054714,2018-05-28,2015-08-29,KAT,1,01 - TOP,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,Unknown,V,40,N,NaN,5,May
1231884,1054714,2018-06-28,2015-08-29,KAT,1,01 - TOP,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,1,ES,Unknown,V,40,N,NaN,6,June
1586303,1054714,2018-07-28,2015-08-29,KAT,1,01 - TOP,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Unknown,V,40,N,NaN,7,July
2047525,1054714,2018-08-28,2015-08-29,KAT,1,01 - TOP,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Unknown,V,40,N,NaN,8,August
2327704,1054714,2018-09-28,2015-08-29,KAT,1,01 - TOP,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Unknown,V,40,N,NaN,9,September
2676369,1054714,2018-10-28,2015-08-29,KAT,1,01 - TOP,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Unknown,V,41,N,NaN,10,October


### Sustitución de region 'Unknowns'

Vamos a sustituir region_code por 28 en estos casos, al ser Madrid la región más repetida, y por 00 en los casos de fuera de España

In [307]:
# Verificar las categorías actuales
print("Categorías actuales:", df_full['region_code'].cat.categories)

# Agregar nuevas categorías
df_full['region_code'] = df_full['region_code'].cat.add_categories(['28', '00'])

# Aplicar las actualizaciones
df_full.loc[(df_full["region_code"] == 'Unknown') & (df_full["country_id"] == 'ES'), "region_code"] = '28'
df_full.loc[(df_full["region_code"] == 'Unknown') & (df_full["country_id"] != 'ES'), "region_code"] = '00'

# Verifica si quedan valores 'Unknown'
unknown_rows = df_full[df_full["region_code"] == 'Unknown']

unknown_rows

Categorías actuales: Index([      1.0,       2.0,       3.0,       4.0,       5.0,       6.0,
             7.0,       8.0,       9.0,      10.0,      11.0,      12.0,
            13.0,      14.0,      15.0,      16.0,      17.0,      18.0,
            19.0,      20.0,      21.0,      22.0,      23.0,      24.0,
            25.0,      26.0,      27.0,      28.0,      29.0,      30.0,
            31.0,      32.0,      33.0,      34.0,      35.0,      36.0,
            37.0,      38.0,      39.0,      40.0,      41.0,      42.0,
            43.0,      44.0,      45.0,      46.0,      47.0,      48.0,
            49.0,      50.0,      51.0,      52.0, 'Unknown'],
      dtype='object')


,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition


### Sustitución Region_Codes - Provincia

In [308]:
# Diccionario de mapeo de códigos de provincia a nombres de provincia

provincia_mapping = {
    00: "Extranjero",1: "Álava",2: "Albacete",3: "Alicante",4: "Almería",5: "Ávila",6: "Badajoz",7: "Baleares, Islas",
    8: "Barcelona",9: "Burgos",10: "Cáceres",11: "Cádiz",12: "Castellón",13: "Ciudad Real",14: "Córdoba",15: "Coruña, A",
    16: "Cuenca",17: "Girona",18: "Granada",19: "Guadalajara",20: "Gipuzkoa",21: "Huelva",22: "Huesca",23: "Jaén",24: "León",
    25: "Lleida",26: "Rioja, La",27: "Lugo",28: "Madrid",29: "Málaga",30: "Murcia",31: "Navarra",32: "Ourense",33: "Asturias",
    34: "Palencia",35: "Palmas, Las",36: "Pontevedra",37: "Salamanca",38: "Santa Cruz de Tenerife",39: "Cantabria",40: "Segovia",
    41: "Sevilla",42: "Soria",43: "Tarragona",44: "Teruel",45: "Toledo",46: "Valencia",47: "Valladolid",48: "Bizkaia",49: "Zamora",
    50: "Zaragoza",51: "Ceuta",52: "Melilla"}

In [309]:
# Mapear los códigos de 'region_code' a nombres de provincia
df_full['region_code'] = df_full['region_code'].map(provincia_mapping)

# Verificar que la sustitución se realizó correctamente
print(df_full[['region_code']].drop_duplicates())

                    region_code
1479563                  Madrid
3628238               Barcelona
237926                  Sevilla
237918                 Valencia
237937                     León
4480619               Cantabria
237940                 Alicante
1479538                 Navarra
1479554               Coruña, A
237904                 Albacete
3628190                  Toledo
2943585                 Segovia
237913                   Murcia
237911                 Zaragoza
2168148             Palmas, Las
5132997             Ciudad Real
4876072                  Huelva
4876076                    Lugo
237978               Valladolid
237969                  Badajoz
3839358              Pontevedra
237993                Salamanca
2943744                  Málaga
2943675                 Córdoba
2977050               Castellón
1222898                 Granada
237822                   Cuenca
1479760             Guadalajara
237839                 Asturias
1479744                  Burgos
3033993 

In [310]:
# Sustituir los valores NaN en 'region_code' por 'extranjero'
df_full['region_code'] = df_full['region_code'].fillna('Extranjero')
df_last_partition['region_code'] = df_full['region_code']

# Verificar que la sustitución se realizó correctamente
print(df_full['region_code'].unique())

['Madrid' 'Barcelona' 'Sevilla' 'Valencia' 'León' 'Cantabria' 'Alicante'
 'Navarra' 'Coruña, A' 'Albacete' 'Toledo' 'Segovia' 'Murcia' 'Zaragoza'
 'Palmas, Las' 'Ciudad Real' 'Huelva' 'Lugo' 'Valladolid' 'Badajoz'
 'Pontevedra' 'Salamanca' 'Málaga' 'Córdoba' 'Castellón' 'Granada'
 'Cuenca' 'Guadalajara' 'Asturias' 'Burgos' 'Bizkaia' 'Lleida' 'Cádiz'
 'Baleares, Islas' 'Soria' 'Cáceres' 'Girona' 'Melilla' 'Extranjero'
 'Jaén' 'Almería' 'Tarragona' 'Rioja, La' 'Santa Cruz de Tenerife'
 'Teruel' 'Zamora' 'Gipuzkoa' 'Ourense' 'Álava' 'Ávila' 'Huesca'
 'Palencia' 'Ceuta']


In [311]:
# Datos corregidos con las coordenadas de las ciudades
data_with_coordinates = {
    'municipality': ['Madrid', 'Barcelona', 'Sevilla', 'Valencia', 'León', 'Cantabria', 'Alicante', 'Navarra',
                     'Coruña, A', 'Albacete', 'Toledo', 'Segovia', 'Murcia', 'Zaragoza', 'Palmas, Las',
                     'Ciudad Real', 'Huelva', 'Lugo', 'Valladolid', 'Badajoz', 'Pontevedra', 'Salamanca',
                     'Málaga', 'Córdoba', 'Castellón', 'Granada', 'Cuenca', 'Guadalajara', 'Asturias',
                     'Burgos', 'Bizkaia', 'Lleida', 'Cádiz', 'Baleares, Islas', 'Soria', 'Cáceres',
                     'Girona', 'Melilla', 'Jaén', 'Almería', 'Tarragona', 'Rioja, La',
                     'Santa Cruz de Tenerife', 'Teruel', 'Zamora', 'Gipuzkoa', 'Ourense', 'Álava', 'Ávila',
                     'Huesca', 'Palencia', 'Ceuta'],
    'lat': [40.4168, 41.3851, 37.3886, 39.4699, 42.6030, 43.1833, 38.3452, 42.8154, 43.3623, 39.0377,
            39.8560, 40.9570, 37.9922, 41.6488, 28.1212, 39.0022, 37.2625, 43.0110, 38.8799, 42.2636,
            40.9700, 37.7820, 39.9833, 38.2480, 40.0731, 40.6502, 43.2564, 42.3404, 40.8120, 39.5644,
            39.5638, 41.9773, 39.5696, 41.7654, 40.3809, 42.1441, 35.2891, 90.0000, 37.7692, 36.8340,
            41.1180, 42.4650, 28.4636, 40.3450, 41.5040, 43.3557, 42.8500, 42.0056, 40.6572, 42.1655,
            39.9560, 35.8896],
    'lon': [-3.7038, 2.1734, -5.9823, -0.3763, -5.5676, -3.8002, -0.4810, -1.6461, -8.4115, -1.8587,
            -4.0273, -4.4771, -1.1307, -0.8765, -15.4135, -3.9290, -6.9430, -8.6469, -6.9703, -8.5977,
            -4.4940, -4.7821, -0.0416, -4.6452, -1.3776, -3.8002, -0.6295, -4.1661, -0.9734, -0.8160,
            -4.5048, 0.6200, 2.6502, -3.7038, -2.0740, 2.8248, 4.4213, 0.0000, -3.7891, -2.4560,
            1.2456, -2.4684, -16.2500, -1.1262, -5.7450, -1.5152, -7.8647, -3.8355, -4.7005, -2.8723,
            -4.5491, -5.3118]
}

# Crear un DataFrame con los datos
city_coordinates = pd.DataFrame(data_with_coordinates)

# Contar la frecuencia de cada ciudad en 'region_code' de df_last_partition
city_data = df_last_partition['region_code'].value_counts().reset_index()
city_data.columns = ['municipality', 'count']

# Fusionar los datos de conteo con las coordenadas
merged = pd.merge(city_data, city_coordinates, on='municipality', how='left')

# Crear el mapa de puntos con círculos sólidos
fig = px.scatter_mapbox(
    merged,
    lat='lat',
    lon='lon',
    size='count',  # Tamaño de los puntos basado en el conteo
    color='count',  # Color basado en el conteo
    size_max=30,  # Tamaño máximo de los círculos
    center={"lat": 40.4168, "lon": -3.7038},
    zoom=5,
    title='Distribución de Usuarios en España',
    color_continuous_scale='Viridis'
)

# Ajustar el tamaño del gráfico
fig.update_layout(
    mapbox_style="carto-positron",
    width=1000,  # Ancho del gráfico
    height=800  # Altura del gráfico
)

# Mostrar el gráfico interactivo
fig.show()

## SALARY

In [312]:
# Contar cuántos valores únicos de 'salary' tiene cada cliente
estado_por_cliente_salary = df_full.groupby('pk_cid')['salary'].nunique()

# Identificar clientes con múltiples valores de salario (cambios en el salario)
clientes_mixtos_salary = estado_por_cliente_salary[estado_por_cliente_salary > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_salary = df_full[~df_full['pk_cid'].isin(clientes_mixtos_salary)]

# Identificar clientes que tienen siempre el mismo salario
clientes_mismo_salary = df_no_mixtos_salary.groupby('pk_cid')['salary'].nunique()
clientes_mismo_salary = clientes_mismo_salary[clientes_mismo_salary == 1].index

# Identificar clientes con valores NaN en 'salary' (excluyendo clientes mixtos)
clientes_con_nan_salary = df_no_mixtos_salary[df_no_mixtos_salary['salary'].isna()]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_mismo_salary = len(clientes_mismo_salary)
n_con_nan_salary = len(clientes_con_nan_salary)
n_mixtos_salary = len(clientes_mixtos_salary)

# Mostrar los resultados
print(f"Clientes que tienen siempre el mismo salario: {n_mismo_salary}")
print(f"Clientes con valores NaN en 'salary' en todas las ingestas: {n_con_nan_salary}")
print(f"Clientes con valores de salario mixtos, varían con la ingesta de datos (diferentes valores): {n_mixtos_salary}")

Clientes que tienen siempre el mismo salario: 299443
Clientes con valores NaN en 'salary' en todas las ingestas: 156930
Clientes con valores de salario mixtos, varían con la ingesta de datos (diferentes valores): 0


In [313]:
df_full["salary"].describe()

count    4,450,821.000
mean       115,816.724
std        199,551.889
min          1,202.730
25%         61,500.630
50%         88,654.650
75%        131,669.910
max     28,894,395.510
Name: salary, dtype: float64

### Imputación de Nan

In [314]:
# Filtrar los registros donde 'salary' es NaN
casos_salary_nan = df_full[df_full['salary'].isna()]

casos_salary_nan["country_id"].value_counts()

country_id
ES    1509864
GB        420
US        231
DE        213
FR        212
CH        196
BR         88
BE         86
VE         79
IE         68
MX         65
PL         60
AT         51
AR         51
IT         45
MA         34
CN         28
RU         17
SA         17
LU         17
CI         17
QA         17
DO         17
CL         17
GT         17
ET         17
MR         17
SN         17
GA         17
CO         17
NO         17
CM         17
SE         16
PT         11
DJ         11
JM         11
HU          8
Name: count, dtype: int64

#### Creación Grupos de Edad

Como tenemos valores nulos de practicamente todos los países, se me ocurre aplicar el salario mediano pero atendiendo al grupo de edad del sujeto, país de residencia y actividad comercial.  Por tanto, en primer lugar vamos a segmentar en grupos de edad. La segmentación de edades más utilizada suele ser la siguiente:

Menores: eGrupo compuesto por personas menores de dieciocho años

Jóvenes: El grupo de jóvenes son las personas mayores a dieciocho años y menores de veinticinco años.

Adultos jóvenes: Este segmento incluye personas mayores de veinticinco años y menores de cuarenta años.

Adultos: Son las personas mayores de cuarenta años y menores a los cincuenta y cinco años.

Adultos mayores: Encontramos en este grupo a las personas mayores de cincuenta y cinco años y menores de sesenta y cinco.

Ancianos: Son las personas con más sesenta y cinco años y menos de setenta y cinco años.

Longevos: Son todas las personas de setenta y cinco años en adelante.

In [315]:
# Definir función para categorizar según grupos de edad
def categorizar_edad(edad):
    if edad < 18:
        return 'Menores'
    elif 18 <= edad < 25:
        return 'Jóvenes'
    elif 25 <= edad < 40:
        return 'Adultos jóvenes'
    elif 40 <= edad < 55:
        return 'Adultos'
    elif 55 <= edad < 65:
        return 'Adultos mayores'
    elif 65 <= edad < 75:
        return 'Ancianos'
    else:
        return 'Longevos'

# Crear nueva columna 'grupo_edad' en el DataFrame
df_full['grupo_edad'] = df_full['age'].apply(categorizar_edad)

In [316]:
df_full['grupo_edad'].value_counts()

grupo_edad
Jóvenes            2933039
Adultos jóvenes    1963573
Adultos             714564
Adultos mayores     183589
Ancianos             80234
Longevos             52072
Menores              35853
Name: count, dtype: int64

In [317]:
# Eliminar filas donde 'pk_cid' esté duplicado, quedándote solo con la primera aparición de cada 'pk_cid'
df_full_sin_duplicados = df_full.drop_duplicates(subset=['pk_cid'], keep='first')

In [318]:
df_full_sin_duplicados.shape

(456373, 30)

In [319]:
df_full_sin_duplicados['grupo_edad'].value_counts()

grupo_edad
Jóvenes            226902
Adultos jóvenes    143328
Adultos             57788
Adultos mayores     14889
Ancianos             6534
Longevos             4137
Menores              2795
Name: count, dtype: int64

In [320]:
# Filtrar el DataFrame para incluir solo aquellos con 'grupo_edad' igual a 'menores'
df_menores = df_full_sin_duplicados[df_full_sin_duplicados['grupo_edad'] == 'Menores']

# Calcular la mediana de la columna 'salary' para los menores
mediana_salary_menores = df_menores['salary'].median()

mediana_salary_menores

126388.32

In [321]:
df_menores['salary'].describe()

count       1,901.000
mean      151,454.329
std       105,510.747
min         4,559.550
25%        87,611.160
50%       126,388.320
75%       186,018.390
max     1,344,774.180
Name: salary, dtype: float64

In [322]:
df_menores['salary'].value_counts()

salary
108,644.820    6
273,387.540    4
53,210.940     4
81,222.180     3
140,936.250    3
              ..
118,031.190    1
91,338.630     1
107,360.160    1
71,735.070     1
145,731.120    1
Name: count, Length: 1490, dtype: int64

In [323]:
# Filtrar las filas donde 'salary' no es null
df_menores_sin_na_salary = df_menores[df_menores['salary'].notna()]

df_menores_sin_na_salary

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition,grupo_edad
1476977,617264,2018-07-28,2018-04-09,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Madrid,H,13,N,"286,573.440",7,July,Menores
238555,842350,2018-01-28,2015-12-05,KFA,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Madrid,V,8,N,"275,420.460",1,January,Menores
238647,864627,2018-01-28,2015-06-14,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Málaga,V,16,N,"130,015.800",1,January,Menores
1478197,937368,2018-07-28,2015-01-11,KFA,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Barcelona,H,6,N,"109,631.160",7,July,Menores
83238,970787,2018-01-28,2015-09-05,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Madrid,H,14,N,"234,170.370",1,January,Menores
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493568,1453488,2018-09-28,2018-09-17,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Granada,V,12,N,"60,671.040",9,September,Menores
2493678,1453671,2018-09-28,2018-09-18,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Segovia,V,10,N,"74,837.490",9,September,Menores
2493671,1453707,2018-09-28,2018-09-18,NaN,1,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Madrid,V,4,N,"61,097.040",9,September,Menores
2508202,1453988,2018-09-28,2018-09-18,NaN,0,NaN,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Madrid,V,10,N,"100,846.050",9,September,Menores


Vamos a hacer que la variable grupo edad sea constante para cada individuo, ya que entendemos que un cambio de grupo en un periodo de 3 años no va a suponer un cambio determinante en el patrón de comportamiento de cada cliente, además, si en ese periodo ha cambiado de un segmento a otro, entonces, quiere decir que estaba mucho más próximo al último grupo en cuestión

In [324]:
# Agrupar por 'pk_cid' y obtener el último valor de 'grupo_edad' para cada 'pk_cid'
ultimo_grupo_edad = df_full.groupby('pk_cid')['grupo_edad'].last().reset_index()

# Renombrar la columna 'grupo_edad' en el DataFrame resultante para hacer el merge
ultimo_grupo_edad.rename(columns={'grupo_edad': 'ultimo_grupo_edad'}, inplace=True)

# Hacer un merge del DataFrame original con el DataFrame de los últimos valores
df_full = df_full.merge(ultimo_grupo_edad, on='pk_cid', how='left')

# Actualizar la columna 'grupo_edad' con el último valor registrado
df_full['grupo_edad'] = df_full['ultimo_grupo_edad']

# Eliminar la columna temporal
df_full.drop(columns=['ultimo_grupo_edad'], inplace=True)

In [325]:
df_full['salary'].describe()

count    4,450,821.000
mean       115,816.724
std        199,551.889
min          1,202.730
25%         61,500.630
50%         88,654.650
75%        131,669.910
max     28,894,395.510
Name: salary, dtype: float64

Ahora que tenemos los grupos de edad definidos, podemos proceder a imputar los valores nulos de salary usando la mediana del salario dentro de cada segmento.

In [326]:
# Reemplazar los valores NaN en la columna 'Segment' con '03 - UNIVERSITARIO'
df_full['segment'] = df_full['segment'].fillna('03 - UNIVERSITARIO')

#!!!!(Importante aquí calculamos la mediana sobre el conjunto sin duplicados, si no afecta a la mediana)!!!
# Calcular la mediana de 'salary' para cada combinación de país, región, grupo de edad y actividad comercial
mediana_salary = df_full_sin_duplicados.groupby(['country_id', 'grupo_edad', 'segment', 'region_code'])['salary'].median()

# Imputar los valores nulos de 'salary' con la mediana correspondiente
df_full['salary'] = df_full.apply(
    lambda row: mediana_salary[row['country_id'], row['grupo_edad'], row['segment'], row['region_code']]
    if pd.isna(row['salary']) else row['salary'], axis=1
)

In [327]:
#!!!!(Importante aquí calculamos la mediana sobre el conjunto sin duplicados, si no afecta a la mediana)!!!
# Calcular la mediana de 'salary' para cada combinación de país, grupo de edad y actividad comercial
mediana_salary = df_full_sin_duplicados.groupby(['country_id', 'grupo_edad', 'segment'])['salary'].median()

# Imputar los valores nulos de 'salary' con la mediana correspondiente
df_full['salary'] = df_full.apply(
    lambda row: mediana_salary[row['country_id'], row['grupo_edad'], row['segment']]
    if pd.isna(row['salary']) else row['salary'], axis=1
)

In [328]:
df_full["salary"].describe()

count    5,960,140.000
mean       110,032.635
std        173,419.761
min          1,202.730
25%         65,823.870
50%         87,875.100
75%        125,161.530
max     28,894,395.510
Name: salary, dtype: float64

Categorizar edades:

La función categorizar_edad toma la edad y la clasifica en uno de los grupos definidos.

Luego, apply se utiliza para aplicar esta función a la columna age, creando una nueva columna grupo_edad en el DataFrame.

Calcular mediana:

groupby se utiliza para agrupar los datos por país (country_id), región (region_code), grupo de edad (grupo_edad), y actividad comercial (segment).

La mediana del salario para cada grupo se calcula y se almacena en mediana_salary.

Imputación:

Para cada fila, si el salario es nulo (NaN), se reemplaza con la mediana correspondiente calculada en el paso anterior.

In [329]:
# Filtrar los registros donde 'salary' es NaN
casos_salary_nan = df_full[df_full['salary'].isna()]

# Mostrar los casos con 'salary' como NaN
casos_salary_nan

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,mes_partition,mes_nombre_partition,grupo_edad
6057,130419,2018-02-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,1,0.000,0.000,0,0,0,0,0,US,Extranjero,V,53,N,NaN,2,February,Adultos
6058,130419,2018-03-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,US,Extranjero,V,53,N,NaN,3,March,Adultos
6059,130419,2018-04-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,1,0.000,0.000,0,0,0,0,0,US,Extranjero,V,53,N,NaN,4,April,Adultos
6060,130419,2018-05-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,US,Extranjero,V,53,N,NaN,5,May,Adultos
6061,130419,2018-07-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,US,Extranjero,V,53,N,NaN,7,July,Adultos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962243,1552805,2019-05-28,2019-05-26,NaN,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Madrid,V,2,N,NaN,5,May,Menores
5962284,1552858,2019-05-28,2019-05-26,NaN,1,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Santa Cruz de Tenerife,V,11,N,NaN,5,May,Menores
5962565,1553226,2019-05-28,2019-05-30,NaN,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Sevilla,H,9,N,NaN,5,May,Menores
5962610,1553282,2019-05-28,2019-05-30,NaN,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,ES,Tarragona,H,4,N,NaN,5,May,Menores


Comprobamos que siguen qeudando casos con salary = Nan, aunqueson bastante escasos 1749 filas, que se corresponden con 130 individuos, puede ser que para estos no haya grupos de referencia que cumplan con las mismas características establecidas. Como sigue habiendo datos con Salary Nan, vamos a rellenarlos con el salario mediano del grupo_edad y el country_id

In [330]:
#!!!!(Importante aquí calculamos la mediana sobre el conjunto sin duplicados, si no afecta a la mediana)!!!
# Calcular la mediana del salario por grupo_edad y country_id
median_salary = df_full_sin_duplicados.groupby(['grupo_edad', 'country_id'])['salary'].median().reset_index()

# Renombrar la columna de salario mediano para mayor claridad
median_salary.rename(columns={'salary': 'median_salary'}, inplace=True)

# Combinar la mediana de salario con el DataFrame original
df_full = pd.merge(df_full, median_salary, on=['grupo_edad', 'country_id'], how='left')

# Rellenar los NaN en la columna salary con la mediana correspondiente
df_full['salary'].fillna(df_full['median_salary'], inplace=True)

# Eliminar la columna temporal de salario mediano si ya no es necesaria
df_full.drop(columns=['median_salary'], inplace=True)

# Verificar que no queden valores NaN en salary
nan_count = df_full['salary'].isna().sum()
print(f"Número de valores NaN restantes en 'salary': {nan_count}")

Número de valores NaN restantes en 'salary': 1856


Por último, solo nos quedaremos con la media de country_id al ser una variable que intutivamente más influencia tendrá en los cambios de salario

In [331]:
#!!!!(Importante aquí calculamos la mediana sobre el conjunto sin duplicados, si no afecta a la mediana)!!!
# Calcular la mediana del salario por country_id
median_salary = df_full_sin_duplicados.groupby(['country_id'])['salary'].median().reset_index()

# Renombrar la columna de salario mediano para mayor claridad
median_salary.rename(columns={'salary': 'median_salary'}, inplace=True)

# Combinar la mediana de salario con el DataFrame original
df_full = pd.merge(df_full, median_salary, on=['country_id'], how='left')

# Rellenar los NaN en la columna salary con la mediana correspondiente
df_full['salary'].fillna(df_full['median_salary'], inplace=True)

# Eliminar la columna temporal de salario mediano si ya no es necesaria
df_full.drop(columns=['median_salary'], inplace=True)

# Verificar que no queden valores NaN en salary
nan_count = df_full['salary'].isna().sum()
print(f"Número de valores NaN restantes en 'salary': {nan_count}")

Número de valores NaN restantes en 'salary': 1606


In [332]:
#!!!!(Importante aquí calculamos la mediana sobre el conjunto sin duplicados, si no, afecta a la mediana)!!!
# Calcular la mediana del salario por grupo_edad
median_salary = df_full_sin_duplicados.groupby(['grupo_edad'])['salary'].median().reset_index()

# Renombrar la columna de salario mediano para mayor claridad
median_salary.rename(columns={'salary': 'median_salary'}, inplace=True)

# Combinar la mediana de salario con el DataFrame original
df_full = pd.merge(df_full, median_salary, on=['grupo_edad'], how='left')

# Rellenar los NaN en la columna salary con la mediana correspondiente
df_full['salary'].fillna(df_full['median_salary'], inplace=True)

# Eliminar la columna temporal de salario mediano si ya no es necesaria
df_full.drop(columns=['median_salary'], inplace=True)

# Verificar que no queden valores NaN en salary
nan_count = df_full['salary'].isna().sum()
print(f"Número de valores NaN restantes en 'salary': {nan_count}")

Número de valores NaN restantes en 'salary': 0


In [333]:
df_last_partition['salary'] = df_full['salary']

# Crear nueva columna 'grupo_edad' en df_last_partition
df_last_partition['grupo_edad'] = df_last_partition['age'].apply(categorizar_edad)

# Calcular el salario mediano para cada grupo de edad
salary_by_age_group = df_last_partition.groupby('grupo_edad')['salary'].median().reset_index()
salary_by_age_group.columns = ['grupo_edad', 'salario_median']

# Crear el gráfico de barras
fig = px.bar(salary_by_age_group, 
             x='grupo_edad', 
             y='salario_median', 
             text='salario_median',  # Mostrar el salario mediano en las etiquetas de las barras
             labels={'salario_median':'Salario Mediano', 'grupo_edad':'Grupo de Edad'}, 
             title='Salario Mediano por Grupo de Edad',
             color='salario_median',  # Usar la columna 'salario_median' para variar el color de las barras
             color_continuous_scale='Viridis')  # Usar la paleta de colores personalizada

# Personalizar el formato del texto para mostrar los números enteros
fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

# Ajustar el tamaño del gráfico
fig.update_layout(
    width=1000,  # Ancho del gráfico
    height=800   # Altura del gráfico
)

# Mostrar el gráfico interactivo
fig.show()

## DATASET LOG

In [334]:
# Crear una lista de las columnas que deseas eliminar
columnas_a_eliminar = [
    'deceased', 'entry_date', 'entry_channel', 'active_customer', 
    'segment', 'short_term_deposit', 'loans', 'mortgage', 'funds', 
    'securities', 'long_term_deposit', 'em_account_pp', 'credit_card', 
    'payroll', 'pension_plan', 'payroll_account', 'emc_account', 
    'debit_card', 'em_account_p', 'em_acount'
]

# Crear una copia del DataFrame original y eliminar las columnas especificadas
sociodemographic_df_adrian = df_full.copy()
sociodemographic_df_adrian = sociodemographic_df_adrian.drop(columns=columnas_a_eliminar)

# Verificar las primeras filas del nuevo DataFrame para asegurarse de que las columnas se eliminaron correctamente
sociodemographic_df_adrian.head()

,pk_cid,pk_partition,country_id,region_code,gender,age,salary,mes_partition,mes_nombre_partition,grupo_edad
0,15891,2018-07-28,ES,Madrid,H,59,"102,377.400",7,July,Adultos mayores
1,15891,2018-08-28,ES,Madrid,H,59,"132,384.450",8,August,Adultos mayores
2,16063,2018-11-28,ES,Madrid,H,62,"102,377.400",11,November,Adultos mayores
3,16063,2018-12-28,ES,Madrid,H,62,"132,384.450",12,December,Adultos mayores
4,16063,2019-01-28,ES,Madrid,H,62,"132,384.450",1,January,Adultos mayores


In [335]:
df_full.to_csv('sociodemographic_df_adrian.csv', index=False)

In [336]:
# Columnas de interés
columns_of_interest = ['pk_cid', 'pk_partition', 'country_id', 'region_code', 'gender', 'age', 'salary', 'grupo_edad']

# Comprobar valores NaN
nan_check = sociodemographic_df_adrian[columns_of_interest].isna().sum()

# Comprobar valores 'Unknown'
unknown_check = sociodemographic_df_adrian[columns_of_interest].applymap(lambda x: x == 'Unknown').sum()

# Mostrar resultados
print("Número de valores NaN por columna:")
print(nan_check)

print("\nNúmero de valores 'Unknown' por columna:")
print(unknown_check)

Número de valores NaN por columna:
pk_cid          0
pk_partition    0
country_id      0
region_code     0
gender          0
age             0
salary          0
grupo_edad      0
dtype: int64

Número de valores 'Unknown' por columna:
pk_cid          0
pk_partition    0
country_id      0
region_code     0
gender          0
age             0
salary          0
grupo_edad      0
dtype: int64


In [337]:
# Guardar el DataFrame en un archivo parquet para subirlo a s3 de aws
#sociodemographic_df_adrian.to_parquet('sociodemographic_df_adrian.parquet', index=False)

### DASHBOARD DATASET

In [338]:
# Crear una copia del DataFrame original y eliminar las columnas especificadas
dashboard_df_adrian = sociodemographic_df_adrian.copy()

# 1. Calcular la mediana del salario para cada individuo (pk_cid)
# Asumiendo que la columna de salario se llama 'salary'
mediana_salario = dashboard_df_adrian.groupby('pk_cid')['salary'].median().reset_index()

# Renombrar la columna de salario a 'median_salary'
mediana_salario.rename(columns={'salary': 'median_salary'}, inplace=True)

# 3. Eliminar la columna 'salary' del DataFrame y eliminar la columna 'age' del DataFrame
dashboard_df_adrian = dashboard_df_adrian.drop(columns=['salary'])
dashboard_df_adrian = dashboard_df_adrian.drop(columns=['age'])

# 3. Obtener el último valor de 'region_code' para cada individuo
# Asumiendo que 'pk_partition' es un indicador de tiempo, seleccionaremos la última entrada de cada cliente
ultimo_region_code = dashboard_df_adrian.sort_values('pk_partition').groupby('pk_cid')['region_code'].last().reset_index()

# Unir los datos de mediana de salario y último 'region_code' con los datos originales
# Primero combinamos con la mediana del salario
dashboard_df_adrian = pd.merge(dashboard_df_adrian, mediana_salario, on='pk_cid', how='left')

# Luego combinamos con el último 'region_code'
dashboard_df_adrian = pd.merge(dashboard_df_adrian, ultimo_region_code, on='pk_cid', how='left', suffixes=('', '_latest'))

# Eliminar la columna 'region_code' original, si es necesario
dashboard_df_adrian.drop(columns=['region_code'], inplace=True)

# Renombrar 'region_code_latest' a 'region_code' para claridad
dashboard_df_adrian.rename(columns={'region_code_latest': 'region_code'}, inplace=True)

In [339]:
dashboard_df_adrian.head(20)

,pk_cid,pk_partition,country_id,gender,mes_partition,mes_nombre_partition,grupo_edad,median_salary,region_code
0,15891,2018-07-28,ES,H,7,July,Adultos mayores,"117,380.925",Madrid
1,15891,2018-08-28,ES,H,8,August,Adultos mayores,"117,380.925",Madrid
2,16063,2018-11-28,ES,H,11,November,Adultos mayores,"132,384.450",Madrid
3,16063,2018-12-28,ES,H,12,December,Adultos mayores,"132,384.450",Madrid
4,16063,2019-01-28,ES,H,1,January,Adultos mayores,"132,384.450",Madrid
5,16063,2019-02-28,ES,H,2,February,Adultos mayores,"132,384.450",Madrid
6,16063,2019-03-28,ES,H,3,March,Adultos mayores,"132,384.450",Madrid
7,16063,2019-04-28,ES,H,4,April,Adultos mayores,"132,384.450",Madrid
8,16063,2019-05-28,ES,H,5,May,Adultos mayores,"132,384.450",Madrid
9,16203,2018-12-28,ES,V,12,December,Ancianos,"153,902.880",Barcelona


In [340]:
# Verificar las columnas con más de un valor único para cada pk_cid
variabilidad_por_columna = dashboard_df_adrian.groupby('pk_cid').nunique()

# Filtrar para encontrar las columnas que tienen más de un valor único para algún pk_cid
columnas_con_duplicados = variabilidad_por_columna.columns[variabilidad_por_columna.gt(1).any()]

# Mostrar las columnas con duplicados y algunos ejemplos de duplicados
print(f"Columnas con duplicados para un mismo 'pk_cid': {list(columnas_con_duplicados)}")

Columnas con duplicados para un mismo 'pk_cid': ['pk_partition', 'mes_partition', 'mes_nombre_partition']


In [341]:
# Especificar las columnas a excluir para considerar duplicados
columnas_excluir = ['pk_partition', 'mes_partition', 'mes_nombre_partition']

# Crear una lista de columnas a considerar para eliminar duplicados
columnas_considerar = [col for col in dashboard_df_adrian.columns if col not in columnas_excluir]

# Eliminar duplicados basándose en las columnas especificadas
df_unique = dashboard_df_adrian.drop_duplicates(subset=columnas_considerar)

# Contar los valores únicos de 'pk_cid' en df_unique
num_unicos_df_dash = df_unique['pk_cid'].nunique()

# Contar los valores únicos de 'pk_cid' en df_full
num_unicos_df_full = df_full['pk_cid'].nunique()

print("Número de valores únicos de pk_cid en df_unique es:", num_unicos_df_dash)
print("Número de valores únicos de pk_cid en df_full es:", num_unicos_df_full)

Número de valores únicos de pk_cid en df_unique es: 456373
Número de valores únicos de pk_cid en df_full es: 456373


In [342]:
# Mostrar filas que tienen 'pk_cid' duplicados en df_unique
duplicados_pk_cid = df_unique[df_unique['pk_cid'].duplicated(keep=False)]

# Mostrar los resultados
if not duplicados_pk_cid.empty:
    print("Filas con 'pk_cid' duplicados:")
    display(duplicados_pk_cid)
else:
    print("No hay duplicados en la columna 'pk_cid'.")

No hay duplicados en la columna 'pk_cid'.


In [343]:
dashboard_df_adrian.to_csv('dashboard_sociodemographic.csv', index=False)

# *RESUMEN IMPUTACIONES*

Columnas creadas:

 - Mes_partition y mes_nombre_partition: Creamos un identificador del mes partition, ya que puede ser una variable explicativa en el futuro si existen estacionalidades de meses

 - Grupo_edad: La segmentación de edades más utilizada suele ser la siguiente:

        - Menores: eGrupo compuesto por personas menores de dieciocho años

        - Jóvenes: El grupo de jóvenes son las personas mayores a dieciocho años y menores de veinticinco años.

        - Adultos jóvenes: Este segmento incluye personas mayores de veinticinco años y menores de cuarenta años.

        - Adultos: Son las personas mayores de cuarenta años y menores a los cincuenta y cinco años.

        - Adultos mayores: Encontramos en este grupo a las personas mayores de cincuenta y cinco años y menores de sesenta y cinco.

        - Ancianos: Son las personas con más sesenta y cinco años y menos de setenta y cinco años.

        - Longevos: Son todas las personas de setenta y cinco años en adelante.


Asignaciones de valores en variables:

 - Age:

   - Edad mixta: 

      - Variaciones + 3 años. En este caso, vamos a sustituir todos los valores de Age por el último valor conocido, ya que, a futuro lo más coherente será crear categorías de edad en las que se agrupen distintas décadas, por tanto, las variaciones internas de cada cliente van a ser practicamente irrelevantes (si un cliente pasa de 30-33 no es diferenciador), y además, se sobreentiende que los errores de tipificación se corrigen con el tiempo, por tanto, cuanto más reciente sea una entrada, más probable es que sea correcta.

       - Variacion negativa: De igual forma vamos a sustituir todas las edades de los casos con variaciones negativas por la edad que figure en la última entrada de cada usuario

 - Country_id Mixto: En este caso también vamos a modificar los casos de clientes con un country_id mixto en el DataFrame df_full, y asignarles el último país registrado para su pk_cid, ya que al tratarse solo de 25 clientes, todo parece indicar que se trata de errores en la toma de datos. Además, suponiendo que estos cambios de residencia fueran reales, entonces el salario debería sufrir modificaciones, situación que no se observa en ninguno de los casos. Por tanto, nos vamos a quedar con la información más actualizada de cada individuo.

 - Deceased: Como podemos comprobar, por un lado, la variable deceased tiene una variabilidad practicamente despreciable, por tanto, es lógico pensar que dicha variable no va a tener una capacidad explicativa suficiente.

      No vamos a tener en cuenta la variable deceased para hacer modelos al no tener ninguna capacidad predictiva ni explicativa.

      Además dada su alta correlación con otras variables como el grupo de edad, es prudente elimiarla en los modelos para no caer en problemas de multicolinealidad entre las variables explicativas.

 - Gender:

   - Unknown constante: Vamos a imputar los Unknown y los vamos a cambiar por Hombre, ya que, ambos individuos superan los 70 años (sería extraño que se identiffiquen como género fluido y la variable H es la que presenta mayor proporción, por tanto lo imputaremos a esta).

    - Unknown Mixto: Le imputaremos el último género conocido

    - Géneros Mixtos: También vamos a modificar los casos en lso que el género es mixto, tomando el último valor registrado para cada individuo, ya que como hemos apuntado anteriormente, es lógico pensar que la última entrada de cada cliente es la más actualizada, y por tanto, la correcta.

 - Region_code:

    - Region Unknown: Vamos a sustituir region_code por 28 en estos casos, al ser Madrid la región más repetida, y por en los casos de fuera de España.

      Posteriormente sustituimos cada código d eprovincia por la provincia en cuestión y le colocamos Extranjero a los que son de fuera de España

 - Salary: 

    - Imputación de Nulos: Como tenemos valores nulos de practicamente todos los países, se me ocurre aplicar el salario mediano pero atendiendo al grupo de edad del sujeto, país de residencia, región de residencia y actividad comercial. Esto de forma iterativa, es decir, como hay casos para los que no se encuentra salario mediano se van sacando variables de la ecuación por orden restrictivo, es decir las más restrictivas se quedan. Primero nos quedamos con grupo_edad y el country_id, y después solo con grupo_edad
    !!!!(Importante aquí calculamos la mediana sobre el conjunto sin duplicados, si no afecta a la mediana)!!!